In [ ]:
# Author:  Molly Rudisill
# Date: july 18
# Description: created clusters based on the cleaned and processed data that clusters names according to a base name that is most similar
# Struggles: understanding how to put this in with the other data
# Future plans: i want to be able to score something based on maybe the popularity of the names and if they are the same person or not

In [1]:
import pandas as pd
import numpy as np
import Levenshtein
import re
from itertools import combinations

In [2]:
df = pd.read_csv("/Users/mollyrudisill/Voxcroft/cleandata_processed.csv", index_col="Unnamed: 0")

/var/folders/dk/95f115l12ms7fs6v815_g44c0000gn/T/ipykernel_2482/3905274390.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/mollyrudisill/Voxcroft/cleandata_processed.csv", index_col="Unnamed: 0")


In [3]:
subset_data = df[:10000]

In [4]:
# Function to clean up names
def clean_name(name):
    # Remove leading and trailing spaces
    name = name.strip()
    if name.startswith('"') or name.startswith('@'):
        name = name[1:]
    if name.endswith("'S"):
        name = name[:-2]

    return name

# Clean the 'Name' column
subset_data['Name'] = subset_data['Name'].apply(clean_name)

# Filter out names longer than 30 characterss
subset_data = subset_data[subset_data['Name'].str.len() <= 20]

/var/folders/dk/95f115l12ms7fs6v815_g44c0000gn/T/ipykernel_2482/3543968291.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['Name'] = subset_data['Name'].apply(clean_name)


In [5]:
subset_data

,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,Article,Frequency
0,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,BENNY GANTZ,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['0'],0.0,1.0
1,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,BENJAMIN NETANYAHU,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['1'],0.0,1.0
2,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,YOAV GALLANT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['2'],0.0,1.0
3,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,GADI EISENKOT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['3'],0.0,1.0
4,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,BEZALEL SMOTRICH,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['4'],0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10733,6/7/24 15:29,"After sport, arts and culture minister Zizi Ko...",KODWA,People,timeslive.co.za,NaN,"[Primary: Arts, culture, entertainment and med...","['ANC', 'South Africa']",['Human Rights'],['3'],3449.0,1.0
10734,6/7/24 15:29,"After sport, arts and culture minister Zizi Ko...",EKURHULENI,People,timeslive.co.za,NaN,"[Primary: Arts, culture, entertainment and med...","['ANC', 'South Africa']",['Human Rights'],['4'],3449.0,1.0
10736,6/7/24 15:29,"After sport, arts and culture minister Zizi Ko...",JOHN KANI,People,timeslive.co.za,NaN,"[Primary: Arts, culture, entertainment and med...","['ANC', 'South Africa']",['Human Rights'],['6'],3449.0,1.0
10737,6/7/24 15:29,"After sport, arts and culture minister Zizi Ko...",CONNIE CHIUME,People,timeslive.co.za,NaN,"[Primary: Arts, culture, entertainment and med...","['ANC', 'South Africa']",['Human Rights'],['7'],3449.0,1.0


In [6]:
# Extract unique names
names = subset_data['Name'].unique()

In [7]:
# Initialize distance matrix
distance_matrix = np.zeros((len(names), len(names)))
similarity_matrix = np.zeros((len(names), len(names)))

In [8]:
# Calculate Levenshtein distances and similarity scores
for i, j in combinations(range(len(names)), 2):
    distance = Levenshtein.distance(names[i], names[j])
    max_length = max(len(names[i]), len(names[j]))
    similarity_score = 1 - (distance / max_length)
    
    distance_matrix[i, j] = distance
    distance_matrix[j, i] = distance
    
    similarity_matrix[i, j] = similarity_score
    similarity_matrix[j, i] = similarity_score


In [9]:
similarity_df = pd.DataFrame(similarity_matrix, index=names, columns=names)
similarity_df

,BENNY GANTZ,BENJAMIN NETANYAHU,YOAV GALLANT,GADI EISENKOT,BEZALEL SMOTRICH,ASSAF SHAPIRA,INBAL TZACH,TAL SHOHAM,SMOTRICH,MATHABATHA,...,U-OSIMHEN,OKUHLE NGEZWE,LETHU WUKUTHI,UJOSE PESEIRO,UFINIDI GEORGE.,OSANDA KUSHIYA,SONKE SIYAKUQONDA,LEBOGANG MASHILE,MASHILE,JOHN KANI
BENNY GANTZ,0.000000,0.333333,0.250000,0.076923,0.250000,0.153846,0.090909,0.000000,0.000000,0.090909,...,0.090909,0.153846,0.230769,0.076923,0.200000,0.142857,0.235294,0.250000,0.000000,0.363636
BENJAMIN NETANYAHU,0.333333,0.000000,0.222222,0.222222,0.277778,0.166667,0.277778,0.166667,0.166667,0.166667,...,0.166667,0.111111,0.166667,0.166667,0.166667,0.222222,0.166667,0.166667,0.111111,0.277778
YOAV GALLANT,0.250000,0.222222,0.000000,0.153846,0.125000,0.153846,0.166667,0.250000,0.000000,0.166667,...,0.083333,0.000000,0.076923,0.153846,0.133333,0.142857,0.235294,0.250000,0.166667,0.333333
GADI EISENKOT,0.076923,0.222222,0.153846,0.000000,0.125000,0.076923,0.000000,0.230769,0.076923,0.076923,...,0.153846,0.076923,0.000000,0.230769,0.133333,0.214286,0.176471,0.187500,0.230769,0.153846
BEZALEL SMOTRICH,0.250000,0.277778,0.125000,0.125000,0.000000,0.250000,0.375000,0.312500,0.500000,0.125000,...,0.062500,0.187500,0.187500,0.187500,0.187500,0.187500,0.058824,0.187500,0.187500,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OSANDA KUSHIYA,0.142857,0.222222,0.142857,0.214286,0.187500,0.285714,0.142857,0.285714,0.142857,0.285714,...,0.071429,0.142857,0.214286,0.214286,0.200000,0.000000,0.294118,0.375000,0.285714,0.285714
SONKE SIYAKUQONDA,0.235294,0.166667,0.235294,0.176471,0.058824,0.235294,0.117647,0.176471,0.117647,0.117647,...,0.176471,0.117647,0.176471,0.176471,0.058824,0.294118,0.000000,0.000000,0.058824,0.235294
LEBOGANG MASHILE,0.250000,0.166667,0.250000,0.187500,0.187500,0.187500,0.250000,0.187500,0.125000,0.187500,...,0.187500,0.125000,0.250000,0.250000,0.062500,0.375000,0.000000,0.000000,0.437500,0.312500
MASHILE,0.000000,0.111111,0.166667,0.230769,0.187500,0.307692,0.090909,0.300000,0.250000,0.300000,...,0.222222,0.153846,0.076923,0.153846,0.133333,0.285714,0.058824,0.437500,0.000000,0.000000


In [13]:
# Define threshold
threshold = 0.7

# Find matching pairs
matches = []

for i in range(len(names)):
    for j in range(i + 1, len(names)):
        if similarity_matrix[i, j] >= threshold:
            matches.append((names[i], names[j], similarity_matrix[i, j]))

# Display matching pairs
for match in matches:
    print(f"Match: {match[0]} and {match[1]} with similarity score: {match[2]}")

Match: BENJAMIN NETANYAHU and BENJAMIN NKANYANE with similarity score: 0.7777777777777778
Match: MATHABATHA and MATHATHO with similarity score: 0.7
Match: AJAY GUPTA and ANIL GUPTA with similarity score: 0.7
Match: SAULOS CHILIMA and SAULOS KLAUS CHILIMA with similarity score: 0.7
Match: GRANDI THEUNISSEN and ANDREW THEUNISSEN with similarity score: 0.7058823529411764
Match: JACO BARKHUIZEN and ELSABE BARKHUIZEN with similarity score: 0.7058823529411764
Match: CYRIL RAMAPHOSA and CYRILRAMAPHOSA with similarity score: 0.9333333333333333
Match: CYRIL RAMAPHOSA and CYRIL RAMAPHOSA’S with similarity score: 0.8823529411764706
Match: CYRIL RAMAPHOSA and WILL RAMAPHOSA with similarity score: 0.7333333333333334
Match: NELSON MANDELA and NELSON MANDELA’S with similarity score: 0.875
Match: NELSON MANDELA and NELSON MANDELA BAY with similarity score: 0.7777777777777778
Match: RAMAPHOSA and ”RAMAPHOSA with similarity score: 0.9
Match: CHRIS LIEBENBERG and CHASE LIEBENBERG with similarity score: 0

In [14]:
def find_clusters(names, similarity_matrix, threshold):
    clusters = {}
    for i in range(len(names)):
        root = names[i]
        cluster = [root]
        for j in range(len(names)):
            if i != j and similarity_matrix[i, j] >= threshold:
                cluster.append(names[j])
        clusters[root] = cluster
    return clusters

# Find clusters
clusters = find_clusters(names, similarity_matrix, threshold)

# Print clusters
for root, cluster in clusters.items():
    print(f"Cluster root: {root}")
    for name in cluster:
        print(f" - {name}")

Cluster root: BENNY GANTZ
 - BENNY GANTZ
Cluster root: BENJAMIN NETANYAHU
 - BENJAMIN NETANYAHU
 - BENJAMIN NKANYANE
Cluster root: YOAV GALLANT
 - YOAV GALLANT
Cluster root: GADI EISENKOT
 - GADI EISENKOT
Cluster root: BEZALEL SMOTRICH
 - BEZALEL SMOTRICH
Cluster root: ASSAF SHAPIRA
 - ASSAF SHAPIRA
Cluster root: INBAL TZACH
 - INBAL TZACH
Cluster root: TAL SHOHAM
 - TAL SHOHAM
Cluster root: SMOTRICH
 - SMOTRICH
Cluster root: MATHABATHA
 - MATHABATHA
 - MATHATHO
Cluster root: STAN MATHABATHA
 - STAN MATHABATHA
Cluster root: REUBEN MADADZHE
 - REUBEN MADADZHE
Cluster root: REGGIE MOLOKOMME
 - REGGIE MOLOKOMME
Cluster root: DANNY NDLOVU
 - DANNY NDLOVU
Cluster root: FRANS MOKWELE
 - FRANS MOKWELE
Cluster root: ROSEMARY MOLEPO
 - ROSEMARY MOLEPO
Cluster root: JEREMIAH NGOBENI
 - JEREMIAH NGOBENI
Cluster root: JIMMY MACHAKA
 - JIMMY MACHAKA
Cluster root: AXED MK
 - AXED MK
Cluster root: JABULANI KHUMALO
 - JABULANI KHUMALO
Cluster root: AJAY GUPTA
 - AJAY GUPTA
 - ANIL GUPTA
Cluster root: 

In [15]:
def get_cluster_for_name(name, clusters):
    for root, cluster in clusters.items():
        if name in cluster:
            return cluster
    return None
# Example usage
input_name = 'CYRIL RAMAPHOSA'
cluster = get_cluster_for_name(input_name, clusters)

if cluster:
    print(f"Cluster for '{input_name}':")
    for name in cluster:
        print(f" - {name}")
else:
    print(f"No cluster found for '{input_name}'")

Cluster for 'CYRIL RAMAPHOSA':
 - CYRIL RAMAPHOSA
 - CYRILRAMAPHOSA
 - CYRIL RAMAPHOSA’S
 - WILL RAMAPHOSA


In [17]:
results = pd.DataFrame({
    'Name': names,
    'Most_Similar_Name': [None] * len(names),
    'Similarity_Score': [0] * len(names)
})

# Find the most similar name and score for each name
for i in range(len(names)):
    most_similar_index = np.argmax(similarity_matrix[i, :])
    results.at[i, 'Most_Similar_Name'] = names[most_similar_index]
    results.at[i, 'Similarity_Score'] = similarity_matrix[i, most_similar_index]

# Display the results
results

/var/folders/dk/95f115l12ms7fs6v815_g44c0000gn/T/ipykernel_2482/1176127399.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.6363636363636364' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  results.at[i, 'Similarity_Score'] = similarity_matrix[i, most_similar_index]


,Name,Most_Similar_Name,Similarity_Score
0,BENNY GANTZ,KENNY LANGA,0.636364
1,BENJAMIN NETANYAHU,BENJAMIN NKANYANE,0.777778
2,YOAV GALLANT,GALLANT,0.583333
3,GADI EISENKOT,NGALO ISONTO,0.461538
4,BEZALEL SMOTRICH,SMOTRICH,0.500000
...,...,...,...
4519,OSANDA KUSHIYA,ESANDA KUPHELA,0.714286
4520,SONKE SIYAKUQONDA,WENA ZIYANDA,0.470588
4521,LEBOGANG MASHILE,BRIAN MASHILE,0.687500
4522,MASHILE,MAGASHULE,0.666667
